## Important Information

If you are using this for a tutorial then you will need linux to run this code. `openfermion` relies on that. Sorry!

In [ ]:
import perceval as pcvl
import perceval.components.unitary_components as comp
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from IPython import display
from scipy.linalg import eigh
from scipy.sparse import linalg
from openfermionpyscf import generate_molecular_hamiltonian
from openfermion.chem import MolecularData
from openfermion import get_fermion_operator
from openfermion import symmetry_conserving_bravyi_kitaev
from openfermion import get_sparse_operator

In [ ]:
input = """2
Sample H2 molecule
H 0.3710 0.0 0.0
H -0.3710 0.0 0.0"""

In [ ]:
class VQESolver:
    def __init__(self, input):
        self.loss_values = []
        self.iteration = 0
        self.total_time = 0
        self.params = []
        self.input = input
        self.measured_qs = []
        self.measurement_bases = ["ZZ", "XX"]
        self.groups = None
        self.h_names = None
        self.h_weights = None
        self.identity_offset = 0.0
        self.states = {
            "00": pcvl.BasicState([0, 1, 0, 1, 0, 0]),
            "01": pcvl.BasicState([0, 1, 0, 0, 1, 0]),
            "10": pcvl.BasicState([0, 0, 1, 1, 0, 0]),
            "11": pcvl.BasicState([0, 0, 1, 0, 1, 0])
        }
    def parse_input(self, input_str):
        lines = input_str.strip().split('\n')
        num_atoms = int(lines[0].strip())
        geometry = []
        for line in lines[2:2 + num_atoms]:
            parts = line.split()
            atom = parts[0]
            coords = tuple(float(x) for x in parts[1:])
            geometry.append((atom, coords))
        return geometry

        self.set_hamiltonian(parse_input(input))
        self.real_energy = None



    def set_hamiltonian(self, geometry, basis='sto-3g', multiplicity=1):
        #if geometry is None:
         #   geometry = [('H', (0.,  0., 0.)), ('H', (0., 0., 0.7414))]

        molecule = MolecularData(geometry, basis, multiplicity)
        mol_ham = generate_molecular_hamiltonian(geometry, basis, multiplicity)
        ferm_op = get_fermion_operator(mol_ham)
        qubit_ham = symmetry_conserving_bravyi_kitaev(ferm_op, 4, 2)

        h_weights = []
        h_names = []
        for term in qubit_ham.terms:
            coeff = qubit_ham.terms[term]
            if term:
                pauli_string = ''.join([op[1] for op in term])
                qubit_string = ''.join(['I' for _ in range(2)])
                for op in term:
                    qubit_string = qubit_string[:op[0]] + op[1] + qubit_string[op[0] + 1:]
                h_weights.append(coeff)
                h_names.append(qubit_string)
            else:
                self.identity_offset = coeff

        self.h_names = h_names
        self.h_weights = h_weights

        self.groups = self.group_hamiltonian_terms(h_names, h_weights)

        # Calculate the exact energy
      #  self.real_energy = self.exact_eigensolver(h_names, [round(weight, 4) for weight in h_weights], self.identity_offset)
        qubit_ham = get_sparse_operator(qubit_ham)
        eigs, _ = linalg.eigsh(qubit_ham, k=1, which="SA")
        ground_energy = eigs[0]
        self.real_energy = ground_energy

    def exact_eigensolver(self, h_names, h_weights, identity_offset):
        n = 4  # For a 2-qubit system
        H = np.zeros((n, n), dtype=np.complex128)

        pauli_matrices = {
            'I': np.eye(2),
            'X': np.array([[0, 1], [1, 0]]),
            'Y': np.array([[0, -1j], [1j, 0]]),
            'Z': np.array([[1, 0], [0, -1]])
        }

        def kron_pauli(term):
            matrices = [pauli_matrices[p] for p in term]
            return np.kron(matrices[0], matrices[1])

        for weight, name in zip(h_weights, h_names):
            H += weight * kron_pauli(name)

        # Adding the identity offset
        H += identity_offset * np.eye(n)

        eigenvalues, _ = eigh(H)
        return np.min(eigenvalues)

    def group_hamiltonian_terms(self, h_names, h_weights):
        ZZ_group = []
        XX_group = []

        for name, weight in zip(h_names, h_weights):
            if 'X' in name:
                XX_group.append((weight, name))
            else:
                ZZ_group.append((weight, name))

        return [ZZ_group, XX_group]

    def H(self):
        circ = pcvl.Circuit(2, name="H")
        hada = pcvl.BS.H()
        circ.add((0, 1), hada)
        return circ


    def RY(self, angle):
        circ = pcvl.Circuit(2, name="RY")
        ry = pcvl.BS.Ry(theta=angle, phi_tl=0, phi_bl=0, phi_tr=0, phi_br=0)
        circ.add((0, 1), ry)
        return circ

    def RX(self, angle):
        circ = pcvl.Circuit(2, name="RX")
        rz = pcvl.BS.Rx(theta=angle)
        circ.add((0, 1), rz)
        return circ

    def RZ(self, angle):
        circ = pcvl.Circuit(2, name="RZ")
        rz = pcvl.BS.Rx(theta=0, phi_tl=-angle / 2, phi_bl=angle / 2, phi_tr=0, phi_br=0)
        circ.add((0, 1), rz)
        return circ


    def Anzats(self, lp):
        circ = pcvl.Circuit(6)
        circ.add(1, self.RX(lp[0]))
        circ.add(3, self.RX(lp[1]))
        circ.add(1, self.RZ(lp[2]))
        circ.add(3, self.RZ(lp[3]))
        circ.add(1, self.RX(lp[4]))
        circ.add(3, self.RX(lp[5]))
        circ.add([0, 1, 2, 3, 4, 5], pcvl.PERM([0, 1, 2, 3, 4, 5]))
        circ.add((3, 4), pcvl.BS())
        circ.add([0, 1, 2, 3, 4, 5], pcvl.PERM([0, 1, 2, 3, 4, 5]))
        circ.add((0, 1), pcvl.BS(pcvl.BS.r_to_theta(1 / 3)))
        circ.add((2, 3), pcvl.BS(pcvl.BS.r_to_theta(1 / 3)))
        circ.add((4, 5), pcvl.BS(pcvl.BS.r_to_theta(1 / 3)))
        circ.add([0, 1, 2, 3, 4, 5], pcvl.PERM([0, 1, 2, 3, 4, 5]))
        circ.add((3, 4), pcvl.BS())
        circ.add([0, 1, 2, 3, 4, 5], pcvl.PERM([0, 1, 2, 3, 4, 5]))
        circ.add(1, self.RX(lp[6]))
        circ.add(3, self.RX(lp[7]))
        circ.add(1, self.RZ(lp[8]))
        circ.add(3, self.RZ(lp[9]))
        circ.add(1, self.RX(lp[10]))
        circ.add(3, self.RX(lp[11]))
        return circ

    def append_Hadamard(self, i: int, circ: pcvl.Circuit) -> None:
        if i == 0:
            circ.add((1), self.H())
        if i == 1:
            circ.add((3), self.H())

    def rotate_measurements(self, c: pcvl.Circuit, pauli_string: str):
        new_circuit = c.copy()
        for i, pauli_op in enumerate(pauli_string):
            if pauli_op == "X":
                self.append_Hadamard(i, new_circuit)
        return new_circuit

    def measured_qubits(self, pauli_string: str):
        measured_qubits = []
        for i, pauli_op in enumerate(pauli_string):
            if pauli_op in ['X', 'Y', 'Z']:
                measured_qubits.append(i)
        return measured_qubits

    def minimize_loss(self, lp):
        num_qubits = 2
        input_index = str(np.binary_repr(0, num_qubits))

        for idx, v in enumerate(lp):
            while lp[idx] < 0:
                lp[idx] += 2 * np.pi
            while lp[idx] >= 2 * np.pi:
                lp[idx] -= 2 * np.pi

        averages = []

        for i in range(len(self.measurement_bases)):
            output_dict = {}

            processor = pcvl.Processor("SLOS")
            new_circ = self.rotate_measurements(self.Anzats(lp), self.measurement_bases[i])
            processor.set_circuit(new_circ)
            processor.with_input(pcvl.BasicState([0, 1, 0, 1, 0, 0]))
            shot_num = 10000_000
            sampler = pcvl.algorithm.Sampler(processor)
            remote_job = sampler.sample_count(shot_num)

            avg = 0
            sum_valid_outputs = 0
            for a in range(len(self.states)):
                b = str(np.binary_repr(a, num_qubits))
                sum_valid_outputs += remote_job['results'][self.states[b]]

            for a in range(len(self.states)):
                b = str(np.binary_repr(a, num_qubits))
                output_dict[b] = remote_job['results'][self.states[b]] / sum_valid_outputs

            for j, term in self.groups[i]:
                for output_state in self.states.keys():
                    state = sum(int(output_state[measure]) for measure in self.measured_qubits(term))
                    if state % 2 == 1:
                        avg -= output_dict[output_state] * float(j)
                    else:
                        avg += output_dict[output_state] * float(j)

            averages.append(avg)

        loss = sum(averages) + self.identity_offset
        self.loss_values.append(loss)

        Real_Energy = vqe_solver.real_energy

        self.iteration += 1

        return loss

    def optimize(self, method="cobyla", options={"tol": 1e-4, "maxiter": 800}):
        init_param = [2 * np.pi * np.random.random() for _ in range(12)]
        result = minimize(self.minimize_loss, init_param, method=method, options=options)
        return float(result.fun)


In [ ]:
vqe_solver = VQESolver(input)
geometry = vqe_solver.parse_input(input)

vqe_solver.set_hamiltonian(geometry)

output = vqe_solver.optimize()

print("Optimized Energy:", output)